In [1]:
import numpy as np
import astropy.units as u
import yaml
import utils
import matplotlib.pyplot as plt

from gammapy.data import DataStore
from gammapy.maps import MapAxis, WcsGeom, Map
from gammapy.datasets import MapDataset
from gammapy.makers import MapDatasetMaker, SafeMaskMaker, FoVBackgroundMaker
from gammapy.modeling.models import FoVBackgroundModel, Models
from gammapy.modeling import Fit
from gammapy.irf import Background3D

import warnings
warnings.filterwarnings('ignore')

In [2]:
# loading general parameters
with open("../general_config.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
conf=cfg['conf']
hessera = 'hess1'
muoneff_flag= cfg['muoneff_flag']
muoneff_flag=False

In [3]:
basedir = f'{cfg["FITS_PROD"]}/{hessera}/std_{conf}_fullEnclosure'
ds = DataStore.from_dir(basedir, f'hdu-index-bg-latest-fov-radec.fits.gz', f'obs-index-bg-latest-fov-radec.fits.gz')
table = ds.obs_table


In [15]:
m1=table['ZEN_PNT'] > 47.5
m2=table['ZEN_PNT'] < 52.5
m3=table['MUONEFF'] > 0.085
mask_final = m1&m2

In [16]:
table[mask_final]

ALT_PNT,AZ_PNT,DATE-END,DATE-OBS,DEADC,DEC_OBJ,DEC_PNT,EVENT_COUNT,EVENT_DEC_MEDIAN,EVENT_ENERGY_MEDIAN,EVENT_RA_MEDIAN,EVENT_TIME_MAX,EVENT_TIME_MIN,GLAT_PNT,GLON_PNT,LIVETIME,MUONCORR,MUONEFF,N_TELS,OBJECT,OBS_ID,ONTIME,QUALITY,RA_OBJ,RA_PNT,TELLIST,TIME-END,TIME-OBS,TSTART,TSTOP,ZEN_PNT,BKG_SCALE
float32,float32,bytes10,bytes10,float32,float32,float32,int64,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,int64,bytes43,int64,float32,int64,float32,float32,bytes7,bytes12,bytes12,float64,float64,float32,float32
37.66612,327.96313,2004-01-18,2004-01-18,1.0,22.014444,22.514444,2590,22.47987,0.88573265,83.68085,96158011.99464297,96157131.04546523,-5.5163794,184.13332,881.0,1.1144654,0.094008945,3,Crab Nebula,18361,881.0,1,83.63333,83.63333,"2,3,4",22:34:35.184,22:19:54.184,96157130.0,96158011.0,52.33388,1.0
42.271683,19.53707,2004-01-19,2004-01-19,1.0,22.014444,22.014444,1106,21.997875,0.66370803,83.651665,96234053.8588891,96233140.90604353,-5.7841935,184.5576,913.0,1.0976503,0.09544523,3,Crab Nebula,18371,913.0,1,83.63333,83.63333,"2,3,4",19:41:57.184,19:26:44.184,96233140.0,96234053.0,47.728317,1.0
42.105804,340.01657,2004-01-21,2004-01-21,1.0,22.014444,24.014444,6914,23.940395,0.67612535,83.6524,96414256.47390175,96412625.67059374,-4.711194,182.86267,1631.0,1.1046572,0.095532775,4,Crab Nebula,18415,1631.0,1,83.63333,83.63333,"1,2,3,4",21:45:20.184,21:18:09.184,96412625.0,96414256.0,47.894196,1.0
42.35883,341.01285,2004-01-24,2004-01-24,1.0,22.014444,20.014444,6030,19.938824,0.5556518,83.69092,96672566.94790077,96670906.04572749,-6.8521466,186.25896,1662.0,1.1183223,0.09429551,4,Crab Nebula,18480,1662.0,0,83.63333,83.63333,"1,2,3,4",21:30:30.184,21:02:48.184,96670904.0,96672566.0,47.64117,1.0
38.31176,329.32797,2004-01-26,2004-01-26,1.0,22.014444,23.014444,7207,22.902702,0.8338543,83.64775,96847398.73824835,96845740.34630203,-5.248265,183.70941,1658.0,1.1260396,0.09364254,4,Crab Nebula,18561,1658.0,0,83.63333,83.63333,"1,2,3,4",22:04:22.184,21:36:44.184,96845740.0,96847398.0,51.68824,1.0
39.3017,331.76346,2004-02-13,2004-02-13,1.0,22.014444,24.014444,4081,23.940182,0.808597,83.64628,98397775.96301198,98396220.52652025,-4.711194,182.86267,1556.0,1.1351221,0.09289033,4,Crab Nebula,18872,1556.0,1,83.63333,83.63333,"1,2,3,4",20:43:59.184,20:18:03.184,98396219.0,98397775.0,50.6983,1.0
40.46108,334.73102,2004-02-14,2004-02-14,1.0,22.014444,24.014444,7051,23.95191,0.78764874,83.67541,98483333.15090847,98481650.73628283,-4.711194,182.86267,1683.0,1.1376472,0.09269147,4,Crab Nebula,18887,1683.0,1,83.63333,83.63333,"1,2,3,4",20:29:57.184,20:01:54.184,98481650.0,98483333.0,49.53892,1.0
39.96587,333.3308,2004-02-15,2004-02-15,1.0,22.014444,24.014444,4876,24.013153,0.8609675,83.6832,98569791.43779135,98568111.64296412,-4.711194,182.86267,1683.0,1.1253082,0.09304583,3,Crab Nebula,18924,1683.0,1,83.63333,83.63333,"1,3,4",20:30:56.184,20:02:53.184,98568109.0,98569792.0,50.03413,1.0
39.676826,332.54654,2004-02-16,2004-02-16,1.0,22.014444,24.514444,1847,24.441086,0.79778713,83.723854,98656119.33778691,98654443.37073112,-4.442268,182.4398,1677.0,1.1527307,0.09145481,4,Crab Nebula,18944,1677.0,1,83.63333,83.63333,"1,2,3,4",20:29:43.184,20:01:46.184,98654442.0,98656119.0,50.323174,1.0
